<a href="https://colab.research.google.com/github/koliby777/pokus-cislo/blob/master/Jazykov%C3%BD_model_CLAUDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Jazykový model CLAUDE**

https://monica.im/share/chat?shareId=QSSCFWegWektmRoB

In [ ]:
import torch
import torch.nn as nn
import math
import nltk
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
!wget https://raw.githubusercontent.com/koliby777/pokus-cislo/master/Texty_EU/cestina.txt
with open('cestina.txt', 'r', encoding='utf-8') as f:
    cz_text = f.read()

!wget https://raw.githubusercontent.com/koliby777/pokus-cislo/master/Texty_EU/anglictina.txt
with open('anglictina.txt', 'r', encoding='utf-8') as f:
    en_text = f.read()

# Kontrola délky stažených textů
print(len(cz_text))
print(len(en_text))


In [2]:
# !pip install nltk


In [ ]:
# nltk.download('punkt')


In [ ]:

# Tokenizace textů
cz_tokens = nltk.word_tokenize(cz_text)
en_tokens = nltk.word_tokenize(en_text)

# Vytvoření slovníku
counter = Counter(cz_tokens + en_tokens)
vocab = sorted(counter, key=counter.get, reverse=True)
vocab_to_idx = {token: idx for idx, token in enumerate(vocab)}

# Převod tokenů na indexy
cz_indices = [vocab_to_idx[token] for token in cz_tokens]
en_indices = [vocab_to_idx[token] for token in en_tokens]

# Vytvoření datových sad
train_data = cz_indices[:int(0.8 * len(cz_indices))] + en_indices[:int(0.8 * len(en_indices))]
val_data = cz_indices[int(0.8 * len(cz_indices)):] + en_indices[int(0.8 * len(en_indices)):]

# Vytvoření datových loaderů
import torch
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(torch.tensor(train_data))
val_dataset = TensorDataset(torch.tensor(val_data))

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

"""
V tomto kódu používáme knihovnu nltk pro tokenizaci textů. Vytvoříme slovník mapující unikátní
tokeny na jejich indexy pomocí Counter a seřadíme tokeny podle jejich frekvence.
Poté převedeme tokeny na jejich indexy pomocí vytvořeného slovníku.
Rozdělíme indexované sekvence na trénovací a validační sady v poměru 80% pro trénování a 20% pro validaci.
Nakonec vytvoříme datové loadery pomocí DataLoader z PyTorch, které nám umožní snadno procházet daty po dávkách během trénování.
Nyní máme připravená data pro trénování našeho jazykového modelu.

"""


In [ ]:
# Kontrola tokenizace
print("První tokeny z českého textu:", cz_tokens[:10])
print("První tokeny z anglického textu:", en_tokens[:10])

# Kontrola slovníku
print("První tokeny ve slovníku:")
for token, idx in list(vocab_to_idx.items())[:10]:
    print(f"{token}: {idx}")

# Kontrola převodu tokenů na indexy
print("První indexy z českých indexovaných sekvencí:", cz_indices[:10])
print("První indexy z anglických indexovaných sekvencí:", en_indices[:10])

# Kontrola datových sad
print("Délka trénovací sady:", len(train_data))
print("Délka validační sady:", len(val_data))

# Kontrola datových loaderů
print("Tvar dávky z trénovacího loaderu:")
for batch in train_loader:
    print(batch[0].shape)
    break

print("Tvar dávky z validačního loaderu:")
for batch in val_loader:
    print(batch[0].shape)
    break


In [ ]:
# Definice modelu

class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_heads, num_layers, hidden_dim, dropout):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.positional_encoding = PositionalEncoding(embedding_dim)
        encoder_layer = nn.TransformerEncoderLayer(embedding_dim, num_heads, hidden_dim, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.fc = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = self.fc(x)
        return x

class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dim, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, embedding_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2).float() * (-math.log(10000.0) / embedding_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x


"""
V této části definujeme architekturu našeho modelu Transformer. Model se skládá z následujících komponent:

Embedding vrstva pro převod indexů tokenů na vektory o zadané dimenzi
Poziční kódování pro zachycení informací o pozici tokenů v sekvenci
Transformer enkodér skládající se z několika vrstev TransformerEncoderLayer
Plně propojená vrstva pro převod výstupu enkodéru na pravděpodobnosti přes slovník
Dále definujeme třídu PositionalEncoding, která implementuje poziční kódování popsané v článku "Attention Is All You Need".
"""


In [ ]:
# Inicializace modelu:

vocab_size = len(vocab_to_idx)
embedding_dim = 256
num_heads = 8
num_layers = 6
hidden_dim = 512
dropout = 0.1

model = TransformerModel(vocab_size, embedding_dim, num_heads, num_layers, hidden_dim, dropout)



In [8]:
# Definice ztrátové funkce a optimizátoru:

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Jako ztrátovou funkci používáme křížovou entropii (CrossEntropyLoss) a jako optimizátor používáme Adam s learning rate 0.001.

In [ ]:
# Trénování modelu:

num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs = batch[0].to(device)
        targets = inputs[:, 1:].contiguous()
        inputs = inputs[:, :-1].contiguous()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")



"""
Zde probíhá trénování modelu po zadaný počet epoch.
V každé epoše procházíme trénovací data po dávkách, provádíme dopředný průchod, počítáme ztrátu,
provádíme zpětnou propagaci a aktualizujeme parametry modelu pomocí optimizátoru.
Na konci každé epochy vypisujeme průměrnou ztrátu.
"""

In [ ]:
# Vyhodnocení modelu:

model.eval()
total_loss = 0
with torch.no_grad():
    for batch in val_loader:
        inputs = batch[0].to(device)
        targets = inputs[:, 1:].contiguous()
        inputs = inputs[:, :-1].contiguous()

        outputs = model(inputs)
        loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
        total_loss += loss.item()

print(f"Validation Loss: {total_loss/len(val_loader):.4f}")
"""
Po trénování vyhodnotíme model na validačních datech.
Procházíme validační data po dávkách, provádíme dopředný průchod a počítáme ztrátu.
Na konci vypisujeme průměrnou ztrátu na validačních datech.
"""

In [ ]:
# Generování textu
def generate_text(model, prompt, max_length=100, temperature=0.7):
    model.eval()
    tokens = nltk.word_tokenize(prompt)
    indices = [vocab_to_idx[token] for token in tokens]
    prompt_tensor = torch.LongTensor(indices).unsqueeze(0).to(device)

    generated_indices = []
    with torch.no_grad():
        for _ in range(max_length):
            outputs = model(prompt_tensor)
            next_token_logits = outputs[-1, :]
            next_token_logits = next_token_logits / temperature
            next_token_probs = torch.softmax(next_token_logits, dim=-1)
            next_token = torch.multinomial(next_token_probs, num_samples=1)
            generated_indices.append(next_token.item())
            prompt_tensor = torch.cat((prompt_tensor, next_token.unsqueeze(0)), dim=1)

    generated_tokens = [list(vocab_to_idx.keys())[idx] for idx in generated_indices]
    generated_text = " ".join(generated_tokens)
    return generated_text


In [ ]:
# Uložení modelu
torch.save(model.state_dict(), "transformer_model.pth")

In [ ]:
# Načtení uloženého modelu
loaded_model = TransformerModel(vocab_size, embedding_dim, num_heads, num_layers, hidden_dim, dropout)
loaded_model.load_state_dict(torch.load("transformer_model.pth"))
loaded_model.eval()